In [1]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Initialize ChromaDB
chroma_db_path = "./grownius_vdbI"
vector_store = Chroma(persist_directory=chroma_db_path, embedding_function=embedding_model)

# Load text and split it into chunks
def store_knowledge(file_path):
    loader = TextLoader(file_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
    docs = text_splitter.split_documents(documents)

    vector_store.add_documents(docs)
    vector_store.persist()
    print(f"✅ Stored {len(docs)} chunks in ChromaDB!")

if __name__ == "__main__":
    store_knowledge("crop_suggestions_grownius.txt")  # Replace with your actual file


/var/folders/s0/smpstk6x2kd_pppyythh4r300000gn/T/ipykernel_56513/1088461914.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/sujairibrahim/ML-Projects/LLMA/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/s0/smpstk6x2kd_pppyythh4r300000gn/T/ipykernel_56513/1088461914.py:16: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2

✅ Stored 528 chunks in ChromaDB!


/var/folders/s0/smpstk6x2kd_pppyythh4r300000gn/T/ipykernel_56513/1088461914.py:27: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load ChromaDB
vector_store = Chroma(persist_directory="./grownius_vdb", embedding_function=embedding_model)

def retrieve_context(query, top_k=3):
    """Retrieves relevant information using similarity search."""
    results = vector_store.similarity_search(query, k=top_k)
    return "\n\n".join([doc.page_content for doc in results]) if results else "No relevant information found."

# Test retrieval
sample_query = "What crops grow well in dry climates?"
print("Retrieved Information:\n", retrieve_context(sample_query))


Retrieved Information:
 *   Water the crop deeply but infrequently to encourage deep root growth
    *   Monitor humidity levels, ideally between 60-80% for healthy growth
    *   Protect the crop from extreme temperature fluctuations (above 35°C or below 15°C)

*   Water the crop deeply but infrequently to encourage deep root growth
    *   Monitor humidity levels, ideally between 60-80% for healthy growth
    *   Protect the crop from extreme temperature fluctuations (above 35°C or below 15°C)

Primary Crop: orange
Input Conditions: {'N': '33', 'P': '14', 'K': '8', 'pH': '7.684420446', 'Rainfall': '110.6823944', 'Humidity': '92.9641969', 'Temperature': '21.03200078'}
LLaMA Suggestions:
**Farming Suggestions**


In [4]:
import os
import json
import requests
from fastapi import FastAPI, Query
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
API_KEY = os.getenv("OPENROUTER_API_KEY")

if not API_KEY:
    raise ValueError("❌ API key not found in .env file!")

# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = Chroma(persist_directory="./grownius_vdb", embedding_function=embedding_model)

app = FastAPI()

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

def retrieve_context(query, top_k=3):
    """Retrieves relevant knowledge from ChromaDB."""
    results = vector_store.similarity_search(query, k=top_k)
    return "\n\n".join([doc.page_content for doc in results]) if results else ""

@app.post("/chat")
async def chat_with_rag(
    feature_1: str = Query(None),
    feature_2: str = Query(None),
    feature_3: str = Query(None),
    feature_4: str = Query(None),
    feature_5: str = Query(None),
    feature_6: str = Query(None),
    feature_7: str = Query(None),
    crop_name: str = Query(None),
    model_prediction: str = Query(None),
    user_query: str = Query(...)
):
    """Handles user queries using retrieved knowledge and OpenRouter's GPT-4o."""
    
    user_input = f"Crop: {crop_name}" if crop_name else "\n".join([feat for feat in [feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7] if feat])
    context = retrieve_context(user_query)

    if not context:
        return {"response": "I'm sorry, but I couldn't find relevant information in the database."}

    prompt = f"""
    You are an AI assistant. Provide responses strictly based on retrieved knowledge.

    ### User Input:  
    {user_input}

    ### Retrieved Knowledge:  
    {context}

    ### Query:  
    {user_query}
    """

    data = {"model": "openai/gpt-4o", "messages": [{"role": "user", "content": prompt}]}
    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, data=json.dumps(data))

    return {"response": response.json()["choices"][0]["message"]["content"]} if response.status_code == 200 else {"error": response.text}


In [5]:
import os
import json
import requests
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
API_KEY = os.getenv("OPENROUTER_API_KEY")

if not API_KEY:
    raise ValueError("❌ API key not found in .env file!")

# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load ChromaDB
vector_store = Chroma(persist_directory="./grownius_vdb", embedding_function=embedding_model)

# Retrieve context from ChromaDB
def retrieve_context(query, top_k=3):
    results = vector_store.similarity_search(query, k=top_k)
    return "\n\n".join([doc.page_content for doc in results]) if results else "No relevant information found."

# Test chatbot retrieval
def test_chatbot():
    sample_query = "N: 45, P: 30, K: 20, pH: 6.5, temperature: 28°C, rainfall: 120mm, humidity: 70%.. and the recommended crop is maize. so give me the best irrigation, fertilizers, seconday crop, additional care tips."
    
    # Retrieve knowledge
    context = retrieve_context(sample_query)

    if not context:
        print("❌ No relevant information found in the database.")
        return

    # Create chatbot prompt
    prompt = f"""
    You are an AI assistant. Provide responses strictly based on retrieved knowledge.

    ### Retrieved Knowledge:  
    {context}

    ### Query:  
    {sample_query}
    """

    # API request
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    data = {"model": "openai/gpt-4o", "messages": [{"role": "user", "content": prompt}]}
    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, data=json.dumps(data))

    # Print response
    if response.status_code == 200:
        print("✅ AI Response:\n", response.json()["choices"][0]["message"]["content"])
    else:
        print("❌ API Error:", response.text)

# Run the test
if __name__ == "__main__":
    test_chatbot()


✅ AI Response:
 ### Farming Suggestions for Maize

#### Best Irrigation Method:
For maize, the best irrigation methods are usually drip or sprinkler systems. These methods efficiently supply water directly to the plant roots, minimizing water wastage and ensuring the crops receive adequate moisture, especially with your conditions of 120mm rainfall and 70% humidity.

#### Recommended Fertilizers:
1. Nitrogen (N) fertilizers are important. Ideally, use urea or ammonium nitrate.
2. Phosphorus (P) can be supplemented with diammonium phosphate (DAP) or triple superphosphate (TSP).
3. Potassium (K) needs can be met using muriate of potash (MOP).

Ensure to apply these fertilizers based on soil test results and local agricultural guidelines to match your conditions of N: 45, P: 30, K: 20, and pH: 6.5.

#### Secondary Crop:
Soybean or sunflower can be recommended as secondary crops with maize. They provide rotational benefits and help with pest control, improving soil health and reducing dise